# Introduction to Linear Algebra

Let's finding eigenvalues (i.e., energy spectrum) and eigenfunctions of the time-independent Schrodinger equation (utilizing [atomic units](https://en.wikipedia.org/wiki/Hartree_atomic_units) where $\hbar=m=1$)
\begin{align}\label{ExactHamiltonianCoordinateRep}
	\left[ -\frac{1}{2}\frac{d^2}{d x^2} + U(x) \right] \psi(x) = E \psi(x).
\end{align}

[**See also derivations in these in-class notes.**](https://wavetulane-my.sharepoint.com/:o:/g/personal/dbondar_tulane_edu/EjZjkvTFp4tNrthBttdxEJQBwxTUBJotcnkl_vWWOoY_XA?e=AjzyDr)

According to the **forward finite difference** approximation,  the derivative of a function $f(x)$ is approximated by

\begin{align}
	f'(x) &\approx [ f(x+\Delta x) - f(x) ] / \Delta x = \left[ f(x) + f'(x) \Delta x + O\left(\Delta x^2 \right) - f(x) \right] / \Delta x
	= f'(x) + O\left( \Delta x \right) \Longrightarrow \label{EqForwardDiffApprox} \\
	f''(x) &\approx [ f'(x+\Delta x) - f'(x) ] / \Delta x  = [ f(x+2\Delta x) - 2f(x + \Delta x) + f(x)] / \Delta x^2 + O\left( \Delta x \right). 
\end{align}

The discretized Schrodinger Eq. reads
\begin{align}\label{ForwardDiffHamiltonianDiscreet}
	-\frac{\psi(x_{k+2}) - 2\psi(x_{k+1})  + \psi(x_{k})}{2\Delta x^2} + U(x_k)  \psi(x_{k}) = E  \psi(x_{k}),
	\qquad k=1,1,2,\ldots,N,
\end{align}
whith the boundary condition $\psi(x_{N+1}) = \psi(x_{N+2}) =0$.

In other words
\begin{align}\label{ForwardDiffHamiltonianCoordinateRep}
	\left[
		\frac{-1}{2 \Delta x^2} 
		\begin{pmatrix}
			1	& -2	& 1 	&	&	& \\
				& 1	& -2	& 1	&	& \\
				&	& \ddots& \ddots & \ddots & \\
				&	&	& 1	& -2	& 1 \\	
				&	&	&	& 1	& -2 \\
				&	&	&	&	& 1 \\
		\end{pmatrix}
		+
		\begin{pmatrix}
			U(x_1)	&	&	&	&	& \\
				& U(x_2) &	&	&	& \\
				& 	& 	& \ddots &	& \\
				&	& 	& 	& U(x_{N-1})  & \\
				&	&	& 	& 	& U(x_{N})
		\end{pmatrix}
	\right]
	\begin{pmatrix}
	 \psi(x_1)\\
	 \psi(x_2) \\
	 \vdots \\
	 \psi(x_{N-1}) \\
	 \psi(x_N) 
	\end{pmatrix}
	= E
	\begin{pmatrix}
	 \psi(x_1)\\
	 \psi(x_2) \\
	 \vdots \\
	 \psi(x_{N-1}) \\
	 \psi(x_N) 
	\end{pmatrix}
 \end{align} 

In [1]:
using LinearAlgebra

$$
E_n = \hbar \omega\left(n + \frac{1}{2}\right),
\qquad n = 0,1,2, \ldots
$$

In [2]:
function energies_forward_fin_diff(U::Function, a::Real, b::Real, N::Int)
    x = range(a, b, N)
    Δx = x[2] - x[1]

    Hamiltonian =  -1 / (2 * Δx^2) * diagm(
        0 => ones(N), 
        1 => -2 * ones(N - 1), 
        2 => ones(N - 2)
    ) 

    Hamiltonian += diagm(U.(x))
        
    return eigvals(Hamiltonian)
end

energies_forward_fin_diff (generic function with 1 method)

In [9]:
energies = energies_forward_fin_diff(
    # potential energy U
    x -> 0.5 * x ^ 2,
    
    # a is the minimal x
    -4,
    
    # b is the max x
    +4,
    
    # N
    10
)

10-element Vector{Float64}:
 -0.5340470679012346
 -0.5340470679012346
  0.25607638888888884
  0.25607638888888884
  1.836323302469136
  1.836323302469136
  4.206693672839506
  4.206693672839506
  7.3671875
  7.3671875

In [4]:
energies[1]

-76.56949625675938

### 😱💩😱💩 backward finite difference is unphysical 😱💩😱💩

Even though the original Schrödinger eq. contains the self-adjoint Hamiltonian, the discretization with the forward difference approximation is non-Hermitian. This is the resource of unphysicallity of the forward and backward finite difference approximations.

## Central finite difference method

The central difference method
\begin{align}
	f'(x) &\approx [ f(x+\Delta x/2) - f(x-\Delta x/2) ] / \Delta x = f'(x) + O\left( \Delta x^2 \right) \Longrightarrow \label{CentralFinitDiffApprox} \\
	f''(x) &\approx  [ f'(x+\Delta x/2) - f'(x-\Delta x/2) ] / \Delta x  = [ f(x + \Delta x) - 2f(x) + f(x- \Delta x)] / \Delta x^2  + O\left( \Delta x^2 \right). 
\end{align}

The discretized Schrodinger Eq. reads
\begin{align}
	-\frac{\psi(x_{k+1}) - 2\psi(x_{k})  + \psi(x_{k - 1})}{2\Delta x^2} + U(x_k)  \psi(x_{k}) = E  \psi(x_{k}),
	\qquad k=1,1,2,\ldots,N,
\end{align}

with the boundary conditions $\psi(x_{-1}) = \psi(x_{N+1}) = 0$, which is a grid representation of $\langle \pm\infty | \psi \rangle = 0$.

This leads to the Hermitian finite dimensional approximation of the Hamiltonian:
\begin{align}
	\left[
		\frac{-1}{2 \Delta x^2} 
		\begin{pmatrix}
			-2	& 1 &	&	& \\
			1	& -2	& 1 &	& \\
				 & \ddots & \ddots & \ddots & \\
				&	& 1	& -2	& 1 \\
				&	&	& 1	& -2
		\end{pmatrix}
		+
		\begin{pmatrix}
			U(x_1)	&	&	&	&	& \\
				& U(x_2) &	&	&	& \\
				& 	& 	& \ddots &	& \\
				&	& 	& 	& U(x_{N-1})  & \\
				&	&	& 	& 	& U(x_{N})
		\end{pmatrix}
	\right]
	\begin{pmatrix}
	 \psi(x_1)\\
	 \psi(x_2) \\
	 \vdots \\
	 \psi(x_{N-1}) \\
	 \psi(x_N) 
	\end{pmatrix}
	= E
	\begin{pmatrix}
	 \psi(x_1)\\
	 \psi(x_2) \\
	 \vdots \\
	 \psi(x_{N-1}) \\
	 \psi(x_N) 
	\end{pmatrix}
 \end{align} 

Thus, the central finite difference preserves the physical structure and it leads to reliable numerical results.

In [11]:
function energies_central_fin_diff(U::Function, a::Real, b::Real, N::Int)
    x = range(a, b, N)
    Δx = x[2] - x[1]

    Hamiltonian =  -1 / (2 * Δx^2) * diagm(
        -1 => ones(N - 1), 
        0 => -2 * ones(N), 
        1 => ones(N - 1), 
    ) 

    Hamiltonian += diagm(U.(x))
        
    return eigvals(Hamiltonian)
end

energies_central_fin_diff (generic function with 1 method)

In [14]:
energies = energies_central_fin_diff(
    # potential energy U
    x -> 0.5 * x ^ 2,
    
    # a is the minimal x
    -4,
    
    # b is the max x
    +4,
    
    # N
    1000
)

1000-element Vector{Float64}:
     0.4999984574712528
     1.5000037672417452
     2.5001647726071012
     3.501562653322186
     4.509162143440161
     5.537903742686995
     6.617339220899079
     7.786132417345963
     9.078354601617708
    10.514874870496273
    12.104914251217947
    13.8513631375914
    15.754588388506116
     ⋮
 31179.021406826567
 31180.764502769358
 31182.3473955413
 31183.76703930244
 31185.018387610282
 31186.087567633498
 31187.00239711077
 31187.466561666093
 31188.716888562263
 31188.731371914215
 31191.282047116205
 31191.28205523517

In [15]:
diff(energies)

999-element Vector{Float64}:
 1.0000053097704924
 1.000161005365356
 1.0013978807150847
 1.007599490117975
 1.0287415992468336
 1.0794354782120843
 1.1687931964468845
 1.2922221842717443
 1.4365202688785654
 1.5900393807216737
 1.7464488863734537
 1.9032252509147156
 2.0595939952831746
 ⋮
 1.901377262740425
 1.7430959427911148
 1.5828927719412604
 1.4196437611390138
 1.2513483078437275
 1.0691800232161768
 0.9148294772712688
 0.46416455532380496
 1.2503268961700087
 0.014483351951639634
 2.5506752019900887
 8.118964615277946e-6